[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/feiglab/peprnallps/blob/main/PeptideRNALLPS.ipynb)

# Analyzing peptide-RNA liquid-liquid phase separation

This notebook analyzes the energetics of peptide-RNA condensates based on radial distribution functions extracted from coarse-grained simulations with the COCOMO model.

In [ ]:
import os
import sys
import numpy as np
import math
import copy
from pathlib import Path

# set to true if running from command line
cmdline=False

if not cmdline:
    import matplotlib.pyplot as plt

The following cell downloads RDFs from github if they are not available.

In [ ]:
if not Path("RDF").exists():
    print('downloading input structures')
    !mkdir -p RDF
    !npx degit feiglab/peprnallps/RDF RDF  

## COCOMO interaction potential

The class below defines the COCOMO interaction potential.

In [ ]:
class Energy:
    def __init__(e,eps,sig,aval,a0):
        e.epsilon=eps          # sqrt(ep1.epsilon*ep2.epsilon)  units: kJ/mol
        e.sigma=sig            # (ep1.sigma+ep2.sigma)/2.0      units: nm
        e.aval=aval            # (ep1.aval*ep2.aval)            units: kJ/mol*nm
        e.a0=a0                # (ep1.a0+ep2.a0)                units: kJ/mol*nm
        e.kappa=1.0            #                                units: nm

    def value(e,r):
        sr=(e.sigma/r)
        sr5=sr*sr*sr*sr*sr
        sr10=sr5*sr5
        shortrange=4.0*e.epsilon*(sr10-sr5)
        longrange=(e.aval+e.a0)/r*np.exp(-r/e.kappa)
        return shortrange+longrange

def combineEpsilon(e1,e2):
    return np.sqrt(e1*e2)

def combineSigma(s1,s2):
    return (s1+s2)*0.5

def combineAval(a1,a2):
    return (a1*a2)

def combineA0val(a01,a02):
    return (a01+a02)
    
def sigFromR(r):
    return 2.0*r*2.0**(-1.0/6.0)

The interaction potentials for specific combinations of interaction sites (Ade, Arg, Gly) are defined below using the combining rules as described in the COCOMO paper.

In [ ]:
radius={}
radius['ade']=0.4220*1.2
radius['gly']=0.2617*1.2
radius['arg']=0.3567*1.2

energy={}
energy['ade_ade']=Energy(0.41,
              sigFromR(radius['ade']),
              combineAval(-0.866,-0.866),
              combineA0val(0.05,0.05))

energy['ade_arg']=Energy(combineEpsilon(0.40,0.41)+0.20,
              combineSigma(sigFromR(radius['ade']),sigFromR(radius['arg'])),
              combineAval(-0.866,0.866),
              combineA0val(0.05,0.05))
energy['arg_ade']=energy['ade_arg']

energy['ade_gly']=Energy(0.41,
              combineSigma(sigFromR(radius['ade']),sigFromR(radius['gly'])),
              combineAval(-0.866,0),
              combineA0val(0.05,0))
energy['gly_ade']=energy['ade_gly']

energy['arg_arg']=Energy(0.40,
              sigFromR(radius['arg']),
              combineAval(0.866,0.866),
              combineA0val(0.05,0.05))

energy['arg_gly']=Energy(combineEpsilon(0.40,0.41),
              combineSigma(sigFromR(radius['arg']),sigFromR(radius['gly'])),
              combineAval(0.866,0),
              combineA0val(0.05,0))
energy['gly_arg']=energy['arg_gly']

energy['gly_gly']=Energy(0.41,
              sigFromR(radius['gly']),
              combineAval(0,0),
              combineA0val(0,0))

### Example monomer interaction potential

Interaction potentials for <span style="color:blue">Arg-Arg</span> (*weakly repulsive*), <span style="color:red">Ade-Ade</span> (*weakly attractive*), and <span style="color:green">Ade-Arg</span> (*strongly attractive*) interactions according to COCOMO. 

Energies are in units of kJ/mol where 'mol' refers to the 'amount' of interaction sites (i.e. 'Ade', 'Arg', 'Gly'). 

In [ ]:
if not cmdline:
    r=np.arange(0.2,5,0.05)
    v=energy['arg_arg'].value(r)
    v2=energy['ade_ade'].value(r)
    v3=energy['ade_arg'].value(r)

    plt.rc('font', size=12)
    plt.figure(figsize=(6,5))

    plt.plot(r,v,color='blue',label="Arg-Arg")
    plt.plot(r,v2,color='red',label="Ade-Ade")
    plt.plot(r,v3,color='green',label="Ade-Arg")
    plt.plot([0,10],[0,0],color='grey',linewidth=2,linestyle='dotted')
    
    plt.xlabel('distance [nm]')
    plt.ylabel('energy [kJ/mol]')
    plt.ylim([-1,1])
    plt.xlim([0,5])
    
    plt.legend()

    plt.show()

## Condensate size and particle densities

The class below manages the information about particle densities inside and outside the condensates according to the simulations. The class also has functions for calculating various system volumes and densities.

In [ ]:
class Density:
    def __init__(d,rna,pep,box=100.0,*,nin=None,nout=None,
                 radval=None,loadrna=None,loadpep=None,loadreplicate=None,altsig=None):
        d.rna=rna          # this is the number of RNA bases
        d.pep=pep          # this is the number of peptide repeats
        
        d.boxsize=box      # this is the size of the simulation box (needed to get the overall system volume)
        
        if nin is None or nout is None or radval is None:
            if loadrna is None:
                loadrna=rna       
            if loadpep is None:
                loadpep=pep
            d.loadClusterComponents(f"a{loadrna}.rgrgg{loadpep}")
        else:
            d.radius=radval

            d.ndens={}
            d.ndens['in']={}
            d.ndens['in']['ade']=nin[0]
            d.ndens['in']['arg']=nin[1]
            d.ndens['in']['gly']=nin[2]
            d.ndens['out']={}
            d.ndens['out']['ade']=nout[0]
            d.ndens['out']['arg']=nout[1]
            d.ndens['out']['gly']=nout[2]
            d.ndens['disperse']={}
            d.ndens['disperse']['ade']=(nin[0]*d.volume('in')+nout[0]*d.volume('out'))/d.sysvol()
            d.ndens['disperse']['arg']=(nin[1]*d.volume('in')+nout[1]*d.volume('out'))/d.sysvol()
            d.ndens['disperse']['gly']=(nin[2]*d.volume('in')+nout[2]*d.volume('out'))/d.sysvol()
            
    def loadClusterComponents(d,sys):
        fname=f"RDF/{sys}/cluster_components.txt"
        c=np.loadtxt(fname)
                    
        d.radius=c[6]     # the radius of the condensate, unit: nm
        d.totdens=c[7]    # the total density 
                
        d.ndens={}
        d.ndens['in']={}    
        d.ndens['in']['ade']=c[0]/d.volume('in')           # the density of 'Ade' inside the condensate
        d.ndens['in']['arg']=c[1]/d.volume('in')           # the density of 'Arg' inside the condensate
        d.ndens['in']['gly']=c[2]/d.volume('in')           # the density of 'Gly' inside the condensate
        d.ndens['out']={}
        d.ndens['out']['ade']=c[3]/d.volume('out')         # the density of 'Ade' outside the condensate
        d.ndens['out']['arg']=c[4]/d.volume('out')         # the density of 'Arg' outside the condensate
        d.ndens['out']['gly']=c[5]/d.volume('out')         # the density of 'Gly' outside the condensate
        d.ndens['disperse']={}    
        d.ndens['disperse']['ade']=(c[0]+c[3])/d.sysvol()  # assuming no condensate, density of 'Ade'   
        d.ndens['disperse']['arg']=(c[1]+c[4])/d.sysvol()  # assuming no condensate, density of 'Arg'
        d.ndens['disperse']['gly']=(c[2]+c[5])/d.sysvol()  # assuming no condensate, density of 'Gly'  

    # this function gives the number of peptide or RNA *molecules* inside/outside condensates or for
    # the entire ('total') system
    
    def nmol(d,state,component):
        if component=='all':
            return d.nmol(state,'pep')+d.nmol(state,'rna')
        else:
            if state=='total':
                if component=='pep':
                    return (d.ndens['in']['arg']*d.volume('in')+d.ndens['out']['arg']*d.volume('out'))/2.0/d.pep
                elif component=='rna':
                    return (d.ndens['in']['ade']*d.volume('in')+d.ndens['out']['ade']*d.volume('out'))/d.rna
            else:
                if component=='pep':
                    return d.ndens[state]['arg']*d.volume(state)/2.0/d.pep
                elif component=='rna':
                    return d.ndens[state]['ade']*d.volume(state)/d.rna
       
    # number of monomers in different states
            
    def number(d,state,component):
        if component=='all':
            return d.number(state,'ade')+d.number(state,'arg')+d.number(state,'gly')
        else:
            if state=='total':
                return d.number('in',component)+d.number('out',component)
            else:
                return d.ndens[state][component]*d.volume(state)
            
    # this function gives the 
    #     volume of the condensate (spherical approximation using the condensate radiues)
    #     volume of the dilute phase (system volume minus condensate)
    #     volume of the disperse phase (system volume)
    #
    # unit: nm^3
    
    def volume(d,state):
        if state=='in':
            return 4.0*math.pi/3.0*(d.radius*d.radius*d.radius)
        elif state=='out':
            return d.sysvol()-d.volume('in')
        elif state=='disperse':
            return d.sysvol()
        else:
            print(f'unknown state {state}')
            return 0.0
    
    # this function gives the system size based on the box size (or an optional alternate box size)
    #
    # unit: nm^3
    
    def sysvol(d,altbox=None):
        if altbox is None:
            return d.boxsize*d.boxsize*d.boxsize
        else:
            return altbox*altbox*altbox
    
    # this function estimates the volume not occupied by peptides or RNA in different states by subtracting 
    # cumulative spherical volumes of the monomer species from the total volume of a given state
    #
    # unit: nm^3
    
    def emptyvol(d,state):
        tvol=d.volume(state)
        adevol=d.ndens[state]['ade']*d.volume(state)*4.0*math.pi/3.0*radius['ade']*radius['ade']*radius['ade']
        argvol=d.ndens[state]['arg']*d.volume(state)*4.0*math.pi/3.0*radius['arg']*radius['arg']*radius['arg']
        glyvol=d.ndens[state]['gly']*d.volume(state)*4.0*math.pi/3.0*radius['gly']*radius['gly']*radius['gly']
        return tvol-adevol-argvol-glyvol

    # this function calculates the molecular volume for RNA or peptides by summing up the spherical
    # volumes of the monomer species
    #
    # unit: nm^3
    
    def molvol(d,component):
        svol=0.0
        if component=='ade' or component=='rna':
            svol=4.0*math.pi/3.0*radius['ade']*radius['ade']*radius['ade']*d.rna
        elif component=='pep':
            svol=4.0*math.pi/3.0*(radius['arg']*radius['arg']*radius['arg']*2.0+
                                  radius['gly']*radius['gly']*radius['gly']*3.0)*d.pep
        return svol
    
    # this gives the molecular density in different states
    #
    # units: 1/nm^3
    
    def moldens(d,tag,state):
        if tag=='ade':
            return (d.ndens[state][tag]/d.rna)
        elif tag=='arg':
            return (d.ndens[state][tag]/(d.pep*2))
        elif tag=='gly':
            return (d.ndens[state][tag]/(d.pep*3))
        elif tag=='pep':
            return (d.ndens[state]['arg']/(d.pep*2))
        else:
            return 0.0        
        

    # entropy of mixing for system
    #
    # units of [kcal/(mol condensate)] 
    
    def smix(d,state):
        temp=300.0                   # temperature in K
        kb=0.001987041*4.184         # Boltzmann constant in kJ/mol
        
        # state
        n=d.nmol(state,'all')
        xpep=d.nmol(state,'pep')/n
        xrna=d.nmol(state,'rna')/n
        
        t1=0.0
        t2=0.0
        if xpep>0:
            t1=xpep*np.log(xpep)
        if xrna>0:
            t2=xrna*np.log(xrna)
        
        return temp*kb*n*(t1+t2)

    # entropy of mixing for system
    #
    # units of [kcal/(mol total peptide)] 
    
    def smixpepmol(d,state):
        return d.smix(state)/d.nmol('total','pep')
    
    # entropy of mixing for system
    #
    # units of [kcal/(mol total aa)] 
    
    def smixaamol(d,state):
        return d.smixpepmol(state)/(d.pep*5.0)



### Example density data

Density/volume data for the polyA20/RGRGG5 system is ready below from the trajectory data.

In [ ]:
dens=Density(20,5)         # read data for polyA20-RGRGG5, feel free to change

print('system volume: %lf [nm^3]' % (dens.sysvol()))
print('total number of peptides: %d' % (dens.nmol('total','pep')))
print('  concentration: %lf [mM]' % (dens.nmol('total','pep')/dens.sysvol()*1E27/6.022E23))
print('total number of RNA: %d' % (dens.nmol('total','rna')))
print('  concentration: %lf [mM]' % (dens.nmol('total','rna')/dens.sysvol()*1E27/6.022E23))
print('')

print('condensate:')
print('  radius: %lf [nm]' % (dens.radius))
print('  volume: %lf [nm^3]' % (dens.volume('in')))
print('  Arg density:   %lf [1/nm^3]' % (dens.ndens['in']['arg']))     # these are the number densities, see SI 
print('  Gly density:   %lf [1/nm^3]' % (dens.ndens['in']['gly']))     # these are the number densities, see SI
print('  Ade density:   %lf [1/nm^3]' % (dens.ndens['in']['ade']))     # these are the number densities, see SI
print('  Number of Arg: %lf' % (dens.number('in','arg')))  
print('  Number of Gly: %lf' % (dens.number('in','gly')))  
print('  Number of Ade: %lf' % (dens.number('in','ade')))  
print('  Number of peptides: %lf' % (dens.nmol('in','pep')))  
print('  Number of RNA:      %lf' % (dens.nmol('in','rna')))  

print('dilute:')
print('  Number of Arg: %lf' % (dens.number('out','arg')))  
print('  Number of Gly: %lf' % (dens.number('out','gly')))  
print('  Number of Ade: %lf' % (dens.number('out','ade')))  
print('  Number of peptides: %lf' % (dens.nmol('out','pep')))  
print('  Number of RNA:      %lf' % (dens.nmol('out','rna')))  


In [ ]:
# peptide repeats for which we have observed condensations at different RNA lenghts

prepeat={}
prepeat['20']=np.array([2,3,4,5])
prepeat['10']=np.array([3,4,5])

# let's initialize some arrays to load data into

v1={}
v2={}
v3={}
for r in '20', '10':
    v1[r]=np.zeros(len(prepeat[r]))
    v2[r]=np.zeros(len(prepeat[r]))
    v3[r]=np.zeros(len(prepeat[r]))
    
# set colors for A20/A10 here
color={}
color['20']='red' 
color['10']='blue'


### Monomer density inside condensate

The density of Arg inside the condensate decreases with shorter peptide length (solid lines below).

The density of Ade inside the condensates increases with shorter peptide length and with longer RNA (dashed lines).

In [ ]:
for r in '20', '10':                                # RNA bases
    for p in range(len(prepeat[r])):                # peptide repeats
        d=Density(int(r),prepeat[r][p])
        v1[r][p]=d.ndens['in']['arg']
        v2[r][p]=d.ndens['in']['ade']
    
if not cmdline:
    plt.rc('font', size=12)
    plt.figure(figsize=(6,5))
    for r in '20', '10':
        plt.plot(prepeat[r]*5,v1[r],color=color[r], label="A%d: Arg" % (int(r)))
        plt.plot(prepeat[r]*5,v2[r],color=color[r], linestyle="dashed", label="A%d: Ade" % (int(r)))
    plt.xlabel('peptide length (residues)')
    plt.ylabel('number density [1/nm^3]')
    plt.legend()
    plt.show()
    
print("RNA bases    Peptide repeats     Arg density [1/nm^3]     Ade density [1/nm^3]")
for r in '20', '10':                                # RNA bases
    for p in range(len(prepeat[r])):                     # peptide repeats
        print("%5d           %5d               %10.5lf              %10.5lf" % 
              (int(r),prepeat[r][p],v1[r][p],v2[r][p]))
    print("")

### Mixing entropy

Mixing entropy is calculated below. -TSmix is always favorable. But the difference between the 'disperse' state and the phase separated state where 'dilute' and 'condensed' phases coexist is positive indicating a different peptide/RNA ratio in the condensate than the overall system. 

Results are shown in units of [kJ/mol] where 'mol' refers to the total number of molecules (any kind) in the system. 

In [ ]:
for r in '20', '10':                                # RNA bases
    for p in range(len(prepeat[r])):                # peptide repeats
        d=Density(int(r),prepeat[r][p])
        ntotal=d.nmol('total','all')
        v1[r][p]=d.smix('total')/ntotal
        v2[r][p]=(d.smix('in')+d.smix('out'))/ntotal
        v3[r][p]=(d.smix('in')+d.smix('out')-d.smix('total'))/ntotal
        
if not cmdline:
    plt.rc('font', size=12)
    plt.figure(figsize=(6,5))
    for r in '20', '10':
        plt.plot(prepeat[r]*5,v1[r],marker='+',markersize=8,color=color[r],linestyle='dashed',linewidth=2,label="disp. A%d" % (int(r)))
        plt.plot(prepeat[r]*5,v2[r],marker='x',markersize=8,color=color[r],linestyle='dotted',linewidth=2,label="sep. A%d" % (int(r)))
        plt.plot(prepeat[r]*5,v3[r],linewidth=3,color=color[r])
    plt.xlabel('Peptide length (residues)')
    plt.ylabel('Mixing entropy -TS [kJ/mol]')
    plt.xlim([4.5,5.5*5])
    plt.ylim([-2,0.5])
    plt.xticks(np.arange(5,25.1,5))
    plt.yticks([-2,-1.5,-1,-0.5,0,0.5])
    plt.plot([0,100],[0,0],linewidth=1,color='black')
    plt.legend()
    plt.show()
        
print("[kJ/mol]")
print("RNA bases    Peptide repeats  -TSmix (disperse)  -TSmix (separated) -TdeltaSmix")
for n in '20', '10':                                # RNA bases
    for p in range(len(prepeat[n])):                # peptide repeats
        print("%5d           %5d           %10.5lf       %10.5lf       %10.5lf" % 
              (int(n),prepeat[n][p],v1[n][p],v2[n][p],v3[n][p]))
    print("")


## Radial distribution functions

The following class manages the radial distribution functions extracted from the simulations.

In [ ]:
import numpy.polynomial.polynomial as poly

class RDF:
    def __init__(r,rna,pep,state,part1,part2,*,
                 radval=None,cut=None,newradius=None,scale=None):
        r.rna=rna                         # number of RNA bases
        r.pep=pep                         # number of peptide repeats
        r.radius=radval                   # condensate radius
        r.sys=f"a{rna}.rgrgg{pep}"        # system name used to find RDF files
        r.p1=part1                        # 'Ade', 'Arg', 'Gly'
        r.p2=part2                        # 'Ade', 'Arg', 'Gly'
        r.s=state                         # 'clus', 'bulk', 'disperse0', 'disperse1' (see below)
        r.rdfr=[]                         # holds data once read from file
        r.rdfavg=[]                       # holds data once read from file
        r.loadRDF(cut,newradius,scale)
            
    def loadRDF(r,cut=None,newrad=None,scale=None):
        fname=f"RDF/{r.sys}/rdf_{r.s}_{r.p1}_{r.p2}.txt"     
        t=np.loadtxt(fname)
        r.rdfr=t[:,0]
        r.rdfavg=t[:,1]
        
        # scaling RDF if requested
        if scale is not None:
            r.rdfavg*=scale
            
        # if cutoff is given, RDF is corrected and normalized to 1 after cutoff 
        # correction is based on Equation S8 (g(r) for a finite sphere) 
        if cut is not None and radius is not None:
            gcut=0.0
            for i in range(len(r.rdfr)):
                if (r.rdfr[i]<=cut):
                    gcut=r.rdfavg[i]            
            d=0
            roots=poly.polyroots([0.5*cut*cut*cut,0.0,-3.0/2.0*cut,1.0-gcut])
            for tr in roots:
                if (tr>0 and abs(tr-r.radius*2.0)<abs(d-r.radius*2.0)):
                    d=tr
                    
            for i in range(len(r.rdfr)):
                if (r.rdfr[i]>cut):
                    r.rdfavg[i]=1.0
                else:
                    t=(r.rdfr[i]/d)
                    r.rdfavg[i]/=(1.0-3.0/2.0*t+0.5*(t*t*t))
                    
            # if new radius is given, RDF is rescaled accordingly
            if newrad is not None:
                for i in range(len(r.rdfr)):
                    t=(r.rdfr[i]/(2.0*newrad))
                    r.rdfavg[i]*=(1.0-3.0/2.0*t+0.5*(t*t*t))     


    # integrate interaction potential over RDF up to given maximum radius
    def integrate(r,ener,maxr=15.0):
        if (len(r.rdfr)<=0):
            return 0.0
        
        dr=r.rdfr[1]-r.rdfr[0]
        n=int(maxr/dr)
        if (int(n/2)*2!=n):
            n+=1
        v=np.zeros(n+1)
        for i in range(n+1):
            rad=r.rdfr[i]
            if (i>1):
                v[i]=r.rdfavg[i]*ener.value(rad)*rad*rad
        tsum=v[0]
        for i in range(1,n,2):
            tsum+=4.0*v[i]
        for i in range(2,n-1,2):
            tsum+=2.0*v[i]
        tsum+=v[n]
        return (4.0*math.pi*tsum*dr/3.0)

### Example radial distribution functions

Scaled and corrected RDFs for Ade-Ade interactions inside the condensate.

Note that the original RDFs were normalized with respect to system (box) volume, but we need RDFs normalized by the condensate volumne when considering the condensate. Therefore, we apply a scaling factor.

Uncorrected RDFs for a finite size sphere decrease as a function of distance. 

Corrected RDFs normalize to 1 at long distances, reflecting an infinite size condensate.

It is possible to rescale the RDFs for a different, finite size radius.

In [ ]:
if not cmdline:
    nrna=20                                   # use data for A20/RGRGG5
    npep=5
    
    dens=Density(nrna,npep)                   # load density data to get cluster size and volume 
    scale=dens.volume('in')/dens.sysvol()     # scale RDFs 
    
    scale=1
    
    #uncorrrected RDF, decreases because of finite size
    rdf_nc=RDF(nrna,npep,'clus','ade','ade',radval=dens.radius,scale=scale)
    
    #corrected RDF, assume infinite size, set to exactly 1 beyond cutoff
    rdf=RDF(nrna,npep,'clus','ade','ade',radval=dens.radius,cut=12,scale=scale)
    
    #corrected and then rescaled to decrease according to larger radius (50 nm)
    rdf_50=RDF(20,5,'clus','ade','ade',radval=dens.radius,cut=12,newradius=50,scale=scale)

    plt.plot(rdf.rdfr,rdf.rdfavg,label='corrected')
    plt.plot(rdf.rdfr,rdf_nc.rdfavg,label='uncorrected')
    plt.plot(rdf.rdfr,rdf_50.rdfavg,label='radius=50nm')
    plt.xlabel('distance [nm]')
    plt.ylabel('g(r)')
    plt.xlim([0,15])
    plt.legend()
    plt.show()

## Energetics of condensate formation

The following class manages the calculation of the condensate energy based on COCOMO using RDFs and densities extracted from the simulations. 

In [ ]:
class ClusterEnergy:
    def __init__(c,rna,pep,state,*,scale=None,rdfcut=None,newrad=None,
                 loadrna=None,loadpep=None,altdensity=None,altbox=None,maxr=None):
        c.rna=rna             # number of RNA bases
        c.pep=pep             # number of peptide repeats
        c.state=state         # 'in', 'disperse' 
                              # 'out' uses densities from dilute phases and RDFs 
                              # from the disperse phase because we do not have good
                              # good dilute phase RDFs.       
        if state=='in':
            tag='clus'
        elif state=='out':
            tag='disperse1'
        elif state=='disperse':
            tag='disperse1'
        else:
            print('unknown state')

        if loadrna is None: loadrna=rna       # we can load RDFs for a different RNA length
        if loadpep is None: loadpep=pep       # we can load RDFs for a different peptide length
            
        if altdensity is None:
            c.density=Density(rna,pep,loadrna=loadrna,loadpep=loadpep)
        else:
            c.density=altdensity              # we can provide alternative condensate density/size information
            
        if altbox is None:
            c.box=c.density.boxsize
        else:
            c.box=altbox                      # we can provide alternate box size
            
        if maxr is None:
            c.maxr=15.0                         # integration radius
        else:
            c.maxr=maxr
        
        # all of the RDFs are read in the following
        
        c.rdf={}
        c.rdf['ade_ade']=RDF(loadrna,loadpep,tag,'ade','ade',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)
        c.rdf['ade_arg']=RDF(loadrna,loadpep,tag,'ade','arg',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)
        c.rdf['arg_ade']=c.rdf['ade_arg']
        c.rdf['ade_gly']=RDF(loadrna,loadpep,tag,'ade','gly',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)
        c.rdf['gly_ade']=c.rdf['ade_gly']
        c.rdf['arg_arg']=RDF(loadrna,loadpep,tag,'arg','arg',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)
        c.rdf['arg_gly']=RDF(loadrna,loadpep,tag,'ade','gly',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)
        c.rdf['gly_arg']=c.rdf['arg_gly']
        c.rdf['gly_gly']=RDF(loadrna,loadpep,tag,'gly','gly',radval=c.density.radius,
                             cut=rdfcut,newradius=newrad,scale=scale)

    # Calculation of enthalpy for a given monomer type. 
    #
    # units: [kJ/mol] *see comment below
    #
    # if result is divided by N_A it gives the enthalpy of one monomer interacting with the rest of the state
    # in units of [kJ]
    #
    # a factor of two is included to account for double-counting of pairwise interactions when summing
    # total enthalpies
    
    def calch(c,component,scale=1):
        hsum=0.0
        for o in ['ade', 'arg', 'gly']:
            tag=f"{component}_{o}"                          # specifies type of interactions, such as 'arg_ade' 
            dens=c.density.ndens[c.state][o]                # number density [1/nm^3] of interacting species 
            h=dens* \
              c.rdf[tag].integrate(energy[tag],c.maxr)/2.0  # enthalpy from convoluting g(r)*density with interaction potential
                                                            # divided by 2 to account for double-counting             
            hsum+=h                                         # contributions from different types of interactions are summed
        return hsum*scale

    # calculation of enthalpy for a given component in an entire molecule
    # this simply multiplies the enthalpy from 'calch' by the number of monomers in a molecule
    # 
    # units are as above
    #
    
    def calchmol(c,component,scale=1):
        hade=c.calch('ade',scale)
        harg=c.calch('arg',scale)
        hgly=c.calch('gly',scale)
        if component=='ade' or component=='rna':
            return hade*c.rna                 # an RNA has 'c.rna' bases
        elif component=='arg':
            return harg*c.pep*2.0             # a peptide has 'c.pep' * 2 'Arg' residues for RGRGGn (c.pep = n)
        elif component=='gly':
            return hgly*c.pep*3.0             # a peptide has 'c.pep' & 3 'Gly' residues for RGRGGn (c.pep =n)
        elif component=='pep':
            return (harg*2.0+hgly*3.0)*c.pep  # a peptide has 'c.pep' & 3 'Gly' and 2 'Arg' residues for RGRGGn
        else:
            return 0.0
    
    # enthalpies from 'calch' multiplied by the number density in a given state
    #
    # this gives the original (total) enthalpy values where the units of energy are [kJ/mol/volume]
    # 
    # this is problematic because:
    # 1) it depends on the state volume (i.e. the condensate volume)
    # 2) the meaning of 'mol' refers to monomers ('Ade', 'Arg', or 'Gly') which cannot be compared to 
    #    any experimental observable
    
    def calchn(c,component):
        h=c.calch(component)
        return h*c.density.ndens[c.state][component]
    
    # enthalpies from 'calch' in units of [kJ/mol] where 'mol' refers to the (experimentally defined) total
    # concentration of peptides
    
    def calchpepmol(c,component):
        s=c.state
        if (component=='rna'):
            return c.calch('ade')*c.rna*c.density.nmol(s,'rna')/c.density.nmol('total','pep')
        elif (component=='pep'):
            return (c.calch('arg')*2.0*c.pep+c.calch('gly')*3.0*c.pep)*c.density.nmol(s,'pep')/c.density.nmol('total','pep')
    
    # enthalpies from 'calch' in units of [kJ/(mol total aa)] where 'mol' refers to the (experimentally defined) total
    # concentration of amino acids (in the peptides)
    
    def calchaamol(c,component):
        return c.calchpepmol(component)/c.pep/5.0
    
    # estimation of confinement entropy (loss of translational motion) based on
    # ratio of accessible volume inside condensate to entire system volume
    # 
    # accessible volume inside condensate is approximated as the molecular volume of a given polymer,
    # under the assumption that a polymer inside the condensate can essentially only 'wiggle' in place
    #
    # what is actually calcuated here is -TS
    #
    # units: [kJ/mol]    where 'mol' refers to a molecule (peptide or RNA) 
    
    def calcs(c,component,scale=1):
        temp=300.0                   # temperature in K
        kb=0.001987041*4.184         # Boltzmann constant in kJ/mol
        vratio=c.density.molvol(component)*scale/c.density.sysvol(c.box)
        #vratio=c.density.emptyvol('in')/c.density.sysvol(c.box)
        return -temp*kb*np.log(vratio)
    
    # rescaling of entropy by molecular density (inside condensate).
    # 
    # this was used in the original energy estimates but gives problematic units
    
    def calcsn(c,component):
        s=c.calcs(component)
        return s*c.density.moldens(component,c.state)
    
    # rescaling of entropy to give energy in units of [kJ/(mol total peptide)]
    #
    # here we use again the ratio of densities of molecules inside the condensate relative to the total density of 
    # peptides in the entire system
    
    def calcspepmol(c,component):
        s=c.calcs(component)
        return s*c.density.nmol(c.state,component)/c.density.nmol('total','pep')
    
    # rescaling of entropy to give energy in units of [kJ/(mol total aa)]
    #
    def calcsaamol(c,component):
        return c.calcspepmol(component)/c.pep/5.0
        

The **total enthalpy** of phase separation is $H_{condensate} + H_{dilute} - H_{disperse}$ 

The enthalpies are in units of [kJ/(mol total amino acids)]. They can be added directly and compared to experiments (from ITC).

In [ ]:
print('                   cond.   dilute  disperse   total')
for r in [10,20]:
    for n in [3,4,5]:
        tcin=ClusterEnergy(r,n,'in')              # use uncorrected RDFs
        tcout=ClusterEnergy(r,n,'out')
        tcd=ClusterEnergy(r,n,'disperse')
        
        totin=tcin.calchmol('rna')*tcin.density.nmol('in','rna') + \
              tcin.calchmol('pep')*tcin.density.nmol('in','pep')
        totout=tcout.calchmol('rna')*tcout.density.nmol('out','rna') + \
               tcout.calchmol('pep')*tcout.density.nmol('out','pep')
        totd=tcd.calchmol('rna')*tcd.density.nmol('disperse','rna') + \
             tcd.calchmol('pep')*tcd.density.nmol('disperse','pep')

        naa=tcin.density.nmol('total','pep')*tcin.pep*5.0

        print("polyA %d RGRGG %1d  %5.3lf + %5.3lf - %5.3lf = %5.3lf [kJ/(mol total aa)]" % \
              (r,n,totin/naa,totout/naa,totd/naa,(totin+totout-totd)/naa))
    print()    

To obtain the **total confinement entropy** for a phase separated system, we have to multiply the entropy per molecule by the number of confined molecules (the ones in the condensate). 

We then normalize by the total number of amino acids as for the enthalpy above.

In [ ]:
for r in [10,20]:
    for n in [3,4,5]:
        tc=ClusterEnergy(r,n,'in')              # use uncorrected RDFs
        srna=tc.calcs('rna')*tc.density.nmol('in','rna')
        spep=tc.calcs('pep')*tc.density.nmol('in','pep')
        naa=tc.density.nmol('total','pep')*tc.pep*5.0
        print("polyA %d RGRGG %1d  RNA: %5.3lf  peptide: %5.3lf   total: %5.3lf [kJ/(mol tot aa)]" % \
              (r,n,srna/naa,spep/naa,(srna+spep)/naa))
    print()    

We are now ready to put everything together and calculate **total free energies** according to G = H - TS.

In [ ]:
for r in [10,20]:
    for n in [3,4,5]:        
        tc=ClusterEnergy(r,n,'in')              # use uncorrected RDFs
        tcout=ClusterEnergy(r,n,'out')
        tcd=ClusterEnergy(r,n,'disperse')
        
        hin=tc.calchmol('rna')*tc.density.nmol('in','rna') + \
            tc.calchmol('pep')*tc.density.nmol('in','pep')
        hout=tcout.calchmol('rna')*tcout.density.nmol('out','rna') + \
             tcout.calchmol('pep')*tcout.density.nmol('out','pep')
        hd=tcd.calchmol('rna')*tcd.density.nmol('disperse','rna') + \
           tcd.calchmol('pep')*tcd.density.nmol('disperse','pep')
        htot=hin+hout-hd
        
        sconf=tc.calcs('rna')*tc.density.nmol('in','rna') + \
              tc.calcs('pep')*tc.density.nmol('in','pep')
        
        smix=tc.density.smix('in')+tc.density.smix('out')-tc.density.smix('total')
        
        stot=sconf+smix
        
        naa=tc.density.nmol('total','pep')*tc.pep*5.0
        print("polyA %d RGRGG %1d  H: %5.3lf  -TS: %5.3lf   G: %5.3lf [kJ/(mol tot aa)]" % \
              (r,n,htot/naa,stot/naa,(htot+stot)/naa))
    print()    

In [ ]:
# these functions extrapolate energies to any RNA or peptide length 
# using RDFs and condensate densities from polyA20/RGRGG4 (or different)

def totalFreeEnergy(nrna,npep,*,cut=None,crad=None,box=None,loadrna=20,loadpep=4):
    ce=ClusterEnergy(nrna,npep,'in',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    ced=ClusterEnergy(nrna,npep,'disperse',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    return (ce.calchaamol('rna')-ced.calchaamol('rna')+
            ce.calchaamol('pep')-ced.calchaamol('pep')+
            ce.calcsaamol('rna')+ce.calcsaamol('pep'))

def totalEnthalpy(nrna,npep,*,cut=None,crad=None,box=None,loadrna=20,loadpep=4):
    ce=ClusterEnergy(nrna,npep,'in',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    ced=ClusterEnergy(nrna,npep,'disperse',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    return (ce.calchaamol('rna')-ced.calchaamol('rna')+
            ce.calchaamol('pep')-ced.calchaamol('pep'))

def totalEntropy(nrna,npep,*,cut=None,crad=None,box=None,loadrna=20,loadpep=4):
    ce=ClusterEnergy(nrna,npep,'in',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    ced=ClusterEnergy(nrna,npep,'disperse',rdfcut=cut,newrad=crad,loadrna=loadrna,loadpep=loadpep,altbox=box)
    return (ce.calcsaamol('rna')+ce.calcsaamol('pep'))

Finally, we put everything together to make figures.

In [ ]:
cut=None; crad=None; box=100

pep=np.array([3,4,5])

h={}; s={}
for r in [10,20]:                  
    h[r]=np.zeros(len(pep)); 
    s[r]=np.zeros(len(pep));    
    for p in range(len(pep)): 
        tc=ClusterEnergy(r,pep[p],"in",rdfcut=cut,newrad=crad,altbox=box)
        tcd=ClusterEnergy(r,pep[p],"disperse")
        h[r][p]=tc.calchaamol('rna')-tcd.calchaamol('rna')+tc.calchaamol('pep')-tcd.calchaamol('pep')
        s[r][p]=tc.calcsaamol('rna')+tc.calcsaamol('pep')    


In [ ]:
if not cmdline:
    import matplotlib.colors as colors
    
    lrna=20      # RNA length for which we load RDFs to make predictions for any length
    lpep=4       # peptide (repeat) length for which we load RDFs
         
    color={}; color[20]='red'; color[10]='blue'
                
    npepval=np.linspace(1,11,11)
    nrnaval=np.linspace(1,21,21)
    X,Y=np.meshgrid(npepval,nrnaval)
    vfunc=np.vectorize(totalFreeEnergy)
    vfunc=np.vectorize(totalEnthalpy)
    vfunc=np.vectorize(totalEntropy)
    
    Zg=totalFreeEnergy(Y,X,cut=cut,crad=crad,box=box,loadrna=lrna,loadpep=lpep)
    Zh=totalEnthalpy(Y,X,cut=cut,crad=crad,box=box,loadrna=lrna,loadpep=lpep)
    Zs=totalEntropy(Y,X,cut=cut,crad=crad,box=box,loadrna=lrna,loadpep=lpep)
    
    plt.rc('font', size=10)
    plt.figure(dpi=300)
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(8,3),layout='constrained') 
    
    for r in [20,10]:
        ax1.plot(pep*5,h[r],marker='+',markersize=8,color=color[r],linestyle='dashed',linewidth=2,label="A%d Hsim" % (r))
        ax1.plot(pep*5,s[r],marker='x',markersize=8,color=color[r],linestyle='dashed',linewidth=2,label="A%d Ssim" % (r))
        ax1.plot(pep*5,h[r]+s[r],linewidth=3,color=color[r],label="A%d Gsim" % (r))
        
    ax1.plot(X[19]*5,Zg[19],linewidth=2,linestyle='dotted',color=color[20],label='A20 G')
    ax1.plot(X[9]*5,Zg[9],linewidth=2,linestyle='dotted',color=color[10],label='A10 G')
    
    ax1.plot(X[19]*5,Zh[19],linewidth=1,linestyle=(0, (2, 8)),color=color[20],label='A20 H')  
    ax1.plot(X[9]*5,Zh[9],linewidth=1,linestyle=(0, (2, 8)),color=color[10],label='A10 H')  
    
    ax1.plot(X[19]*5,Zs[19],linewidth=1,linestyle=(0, (1, 2)),color=color[20],label='A20 -TS')  
    ax1.plot(X[9]*5,Zs[9],linewidth=1,linestyle=(0, (1, 2)),color=color[10],label='A10 -TS')  
       
    ax1.set_xlabel('Peptide length (residues)')
    ax1.set_ylabel('Energy [kJ/mol total peptide residues]')
    ax1.set_xlim([4.5,10.5*5])
    ax1.set_ylim([-4.1,5])
    ax1.set_xticks(np.arange(5,50.1,5))
    ax1.set_yticks([-4,-2,0,2,4])
    ax1.legend(bbox_to_anchor=(1.04,1.1),loc='upper left')
    ax1.plot([0,100],[0,0],linewidth=1,color='black')
    
    cp=ax2.contourf(X*5,Y,Zg,levels=200,cmap='bwr',norm=colors.Normalize(vmin=-12,vmax=12))
    ax2.set_xlabel('Peptide length (residues)')
    ax2.set_ylabel('RNA bases')
    ax2.set_xticks(np.arange(5,50.1,5))
    ax2.set_yticks([1,5,10,15,20])
    ax2.set_xlim([4.5,10.5*5])
    ax2.set_ylim([1,20.8])
        
    cbar=plt.colorbar(cp,ticks=[-2,0,2,4,6,8,10,12,14])
    cbar.ax.set_ylabel('$\Delta G$ [kJ/mol total peptide residues]', rotation=90)

    plt.show()